# Real-time fmri

### timing
- Getting synced time information into the pipeline and logged

- TR start, forp (fiber optic response pad?)--usb--> stimmy ("5" keypress) [set use mode 100 ASCII 12345 (serial) Sends ASCII "B" for blue button, "Y" for yellow, etc.](http://www.curdes.com/technical/932usemodes)
- should go into preprocess.py, and prime for reception of new image
  - "5" trigger arrives indicating acquisition time
  - new image arrives and is bound to the acquisition time
  - stimulus

There is the problem that the preprocess and stimulate may run on separate computers with separate clocks

### reliability
- should test how pub-sub zmq works. is queuing happening? do images get dropped?
- collect --pub-sub--> preprocess --pub-sub--> stimulate [CODE](#pub_sub_reliability)

# Decoding
- should read papers that jack and alex sent out to understand decoding framework

# Test pub-sub
<a id='#pub_sub_reliability'></a>

In [9]:
import zmq

In [45]:
%%writefile publisher.py
#!/usr/bin/env python
import zmq
ctx = zmq.Context()
s = ctx.socket(zmq.PUB)
s.bind('tcp://*:5556')
for i in range(3):
    print 'sending'
    s.send('hi there')


Overwriting publisher.py


In [46]:
%%writefile subscriber.py
#!/usr/bin/env python
import zmq
ctx = zmq.Context()
s = ctx.socket(zmq.SUB)
s.connect('tcp://localhost:5556')
s.setsockopt(zmq.SUBSCRIBE, 'hi')
print 'receiving'
for i in range(3):
    msg = s.recv()
    print msg

Overwriting subscriber.py


In [33]:
r = subprocess.Popen(['python', 'subscriber.py'], stdout=subprocess.PIPE,
                    stderr=subprocess.PIPE)

In [38]:
!python publisher.py

In [37]:
!python subscriber.py

receiving
^CTraceback (most recent call last):
  File "subscriber.py", line 9, in <module>
    print s.recv()
  File "socket.pyx", line 628, in zmq.backend.cython.socket.Socket.recv (zmq/backend/cython/socket.c:5616)
  File "socket.pyx", line 662, in zmq.backend.cython.socket.Socket.recv (zmq/backend/cython/socket.c:5436)
  File "socket.pyx", line 139, in zmq.backend.cython.socket._recv_copy (zmq/backend/cython/socket.c:1771)
  File "checkrc.pxd", line 11, in zmq.backend.cython.checkrc._check_rc (zmq/backend/cython/socket.c:5863)
KeyboardInterrupt

